In [1]:
#install 
!sudo apt install tesseract-ocr
!pip install easyocr
!pip install opencv-python-headless==4.1.2.30
!pip install pdf2image
!apt-get install poppler-utils 
!python -m pip uninstall -y spacy
!python -m pip install -U spacy

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'sudo apt autoremove' to remove it.
The following additional packages will be installed:
  tesseract-ocr-eng tesseract-ocr-osd
The following NEW packages will be installed:
  tesseract-ocr tesseract-ocr-eng tesseract-ocr-osd
0 upgraded, 3 newly installed, 0 to remove and 62 not upgraded.
Need to get 4,795 kB of archives.
After this operation, 15.8 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-eng all 4.00~git24-0e00fe6-1.2 [1,588 kB]
Get:2 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr-osd all 4.00~git24-0e00fe6-1.2 [2,989 kB]
Get:3 http://archive.ubuntu.com/ubuntu bionic/universe amd64 tesseract-ocr amd64 4.00~git2288-10f4998a-2 [218 kB]
Fetched 4,795 kB in 2s (2,978 kB/s)
debconf: unable to initi

In [3]:
import easyocr
import numpy as np
from pdf2image import convert_from_path
import cv2
from PIL import Image
import os
from google.colab.patches import cv2_imshow
import spacy
from spacy.scorer import Scorer
from spacy.tokens import Doc
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm
import json
import cv2
from PIL import Image
import os
import easyocr
import numpy as np
from pdf2image import convert_from_path

In [4]:
path='/content/sample_data/UntitledFolder'
fil=1
fil1=1
reader = easyocr.Reader(['en']) 
for i in os.listdir(path):
    print(i)
    print(i[-3:])
    if(i[-3:]=='PDF' or i[-3:]=='pdf'):
        images = convert_from_path(path+'/'+i,fmt='jpg')
        #print(len(images))
        line=''
        for j in images:
            j.save(str(fil1)+'.png')
            image = cv2.imread(str(fil1)+'.png')
            print(image.shape)
            fil1+=1
            gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
            thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]
        
            # Remove horizontal lines
            horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (50,1))
            detect_horizontal = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, horizontal_kernel, iterations=2)
            cnts = cv2.findContours(detect_horizontal, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            cnts = cnts[0] if len(cnts) == 2 else cnts[1]
            for c in cnts:
                cv2.drawContours(thresh, [c], -1, (0,0,0), 2)
            vertical_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (1,10))
            detected_lines = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, vertical_kernel, iterations=2)
            cnts = cv2.findContours(detected_lines, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            cnts = cnts[0] if len(cnts) == 2 else cnts[1]
            for c in cnts:
                cv2.drawContours(thresh, [c], -1, (0,0,0), 2)
            #cv2_imshow(thresh)
            # Dilate to connect text and remove dots
            kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (10,1))
            dilate = cv2.dilate(thresh, kernel, iterations=2)
            #cv2_imshow(dilate)
            cnts = cv2.findContours(dilate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
            cnts = cnts[0] if len(cnts) == 2 else cnts[1]
            for c in cnts:
                area = cv2.contourArea(c)
                if area < 200:
                    cv2.drawContours(dilate, [c], -1, (255,255,255), -1)

            result = cv2.bitwise_and(image, image, mask=dilate)
            result[dilate==0] = (255,255,255)
            #cv2_imshow(result)
            result1 = reader.readtext(result,detail=0,paragraph=True)
            #print(result1)
            for res in result1:
                line=line+' '+res
        print()
        print(line)
        print()
    else:
        image = cv2.imread(path+'/'+i)
        #print(image.shape)
        gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
        thresh = cv2.threshold(gray, 0, 255, cv2.THRESH_BINARY_INV + cv2.THRESH_OTSU)[1]

        # Remove horizontal lines
        horizontal_kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (50,1))
        detect_horizontal = cv2.morphologyEx(thresh, cv2.MORPH_OPEN, horizontal_kernel, iterations=2)
        cnts = cv2.findContours(detect_horizontal, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cnts = cnts[0] if len(cnts) == 2 else cnts[1]
        for c in cnts:
            cv2.drawContours(thresh, [c], -1, (0,0,0), 2)
        #cv2_imshow(thresh)
        # Dilate to connect text and remove dots
        kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (10,1))
        dilate = cv2.dilate(thresh, kernel, iterations=1)
        #cv2_imshow(dilate)
        cnts = cv2.findContours(dilate, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
        cnts = cnts[0] if len(cnts) == 2 else cnts[1]
        for c in cnts:
            area = cv2.contourArea(c)
            if area < 200:
                cv2.drawContours(dilate, [c], -1, (255,255,255), -1)

        result = cv2.bitwise_and(image, image, mask=dilate)
        result[dilate==0] = (255,255,255)
        #cv2_imshow(result)
        result1 = reader.readtext(result,detail=0,paragraph=True)
        #print(result1)
        line=''
        for res in result1:
            line=line+' '+res
        print()
        print(line)
        print()
    f= open('/content/sample_data/UntitledFolder1/'+str(fil)+".txt","w+")
    for i in line:
        f.write(i)
    f.close()
    print('/content/sample_data/UntitledFolder1/'+str(fil)+".txt")
    fil+=1

financial_document_form 16_2019_11_26-12_44_23_597570_.pdf
pdf
(2339, 1653, 3)
(2339, 1653, 3)
(2339, 1653, 3)
(2339, 1653, 3)
(2339, 1653, 3)
(2339, 1653, 3)

 TDS TRACES Cenualized Proexsing Cell TDS Reconciliation Analysis and Correction Enabling System Governmeni of India Income Tax Department FORM NO. 16 [See rule 31(1)(a)] PART A Certificate under Section 203 of the Income-tax Act; 1961 for tax deducted at source on salary Certificate No.  VKMPMKC Last updated on 16-Jul-2019 Name and address of the Employer Name and address of the Employee INVESTORS CLINIC INFRATECH PRIVATE LIMITED N-5, OPP AXIS BANK, SECTOR-18_ NOIDA 201301 Uttar Pradesh +(91)120-4618200 FINANCEICIPL@GMAILCOM NEERAJ SHARMA B 4 BASMENT, SAXENA HOUSE, SHIVALIK NEAR, MALVIYA NAGAR, DELHI 110017 Delhi Employee Reference No IPAN of the Employee provided by the Employer | (If available) PAN of the Deductor TAN of the Deductor AACCI2147R MRTI00933C ANZPS4822J CIT (TDS) Assessment Year Period with the Employer From To T

In [6]:
path='/content/sample_data/UntitledFolder'
nlp = spacy.blank("en") 
db = DocBin()
for i in os.listdir(path):
    if i[-4:]=='json':
        f = open(path+'/'+i)
        TRAIN_DATA = json.load(f)
        for text, annot in tqdm(TRAIN_DATA['annotations']): 
            doc = nlp.make_doc(text) 
            ents = []
            for start, end, label in annot["entities"]:
                span = doc.char_span(start, end, label=label, alignment_mode="contract")
                if span is None:
                    print("Skipping entity")
                else:
                    ents.append(span)
            doc.ents = ents 
            print(doc.ents)
            db.add(doc)
db.to_disk("val_data.spacy")

100%|██████████| 1/1 [00:00<00:00, 20.62it/s]


(TMXRKFB, 25-Apr-2019, ANANDANILAYAM CHAITANYAKIRAN HOUSING PRIVATE LIMITED 8-3-191/95 II ND FLOOR, LAKSHMI PLAZA, MAIN ROAD, VENGAL RAO NAGAR, HYDERABAD, SOMA SEKHAR GONTLA 3-137, BUDDAVARAM ROAD, GANNAVARAM, KRISHNA 521101 Andhra Pradesh, AALCA5989L, HYDA14052D, AKDPG8805G, 2019-20, 01-Apr-2018 31-Mar-2019, 1944.00, 1944.00, 5,40,000.00, 1,62,618.00)


100%|██████████| 1/1 [00:00<00:00, 83.85it/s]


(AMFCAEL, CHEPO6466F, CXKPS2I94C, 2018-19, 12275.00)


100%|██████████| 1/1 [00:00<00:00, 25.57it/s]


(VLYZDNC, 19-Jul-2019, S P SINGLA CONSTRUCTIONS (P) LTD 1006-1007, 10TH FLOOR, PEARLS BEST HEIGHT-1, N.S.P. PITAMPURA, NEW DELHI, MUKESH KUMAR 501, KANAK BRAJ COMPLEX, BORING ROAD, PATNA, AAGCS5773B, DELS1SS3OE, ASAPK1403F, 2019-20, 01-Apr-2018 31-Mar-2019, 27248.00, 27248.00, 780000.00, 740000.00, 171500.00)


100%|██████████| 1/1 [00:00<00:00, 101.75it/s]


(1440979.00, 1200.00, 1360050.00)


100%|██████████| 1/1 [00:00<00:00, 27.81it/s]

(VKMPMKC, 16-Jul-2019, INVESTORS CLINIC INFRATECH PRIVATE LIMITED N-5, OPP AXIS BANK, SECTOR-18_ NOIDA 201301 Uttar Pradesh, NEERAJ SHARMA B 4 BASMENT, SAXENA HOUSE, SHIVALIK NEAR, MALVIYA NAGAR, DELHI, AACCI2147R, MRTI00933C, ANZPS4822J, 2019-20, 01-Apr-2018 31-Mar-2019, 586576.00, 586576.00, 2919628.00, 2679628.00, 175000.00)


In [ ]:
!zip -r /content/sample_data/UntitledFolder1.zip /content/sample_data/UntitledFolder1

In [ ]:
!python -m spacy download en_core_web_lg

In [ ]:
!python -m spacy init config config.cfg --lang en --pipeline ner --optimize efficiency

In [ ]:
! python -m spacy train config.cfg --output ./ --paths.train /content/training_data.spacy --paths.dev /content/val_data.spacy